In [49]:
import pandas as pd
import numpy as np
import pickle

In [50]:
csitrade=pd.read_csv('ZZ500.csv')
# csitrade.rename(columns={'Close':'close'},inplace=True)

In [51]:
#数据一定要按股票排序
csitrade['close_r']=csitrade['close']/csitrade['close'].shift(1)

In [52]:
csitrade.reset_index(inplace=True,drop=True)

In [53]:
csitrade

,kdcode,dt,close,open,high,low,turnover,volume,close_r
0,000009.SZ,2018-01-02,7.32,7.27,7.33,7.23,140497530.0,19269993.0,NaN
1,000009.SZ,2018-01-03,7.42,7.32,7.44,7.30,170636551.0,23092097.0,1.013661
2,000009.SZ,2018-01-04,7.64,7.52,7.70,7.51,358089946.0,46992707.0,1.029650
3,000009.SZ,2018-01-05,7.55,7.65,7.65,7.51,196259451.0,25990264.0,0.988220
4,000009.SZ,2018-01-08,7.52,7.55,7.63,7.46,178511079.0,23703431.0,0.996026
...,...,...,...,...,...,...,...,...,...
169007,603883.SH,2023-12-25,29.37,29.02,29.58,28.80,101007153.0,3457490.0,1.012061
169008,603883.SH,2023-12-26,29.05,29.26,29.49,28.82,66158144.0,2272587.0,0.989105
169009,603883.SH,2023-12-27,30.08,28.94,30.40,28.80,206474442.0,6939076.0,1.035456
169010,603883.SH,2023-12-28,28.98,30.15,30.20,28.44,264520627.0,9053474.0,0.963431


In [54]:
#大盘和日收益率
import pandas as pd
df_trade=csitrade
df_trade.rename(columns={'kdcode':'instrument','dt':'datetime'},inplace=True)
# df_trade.rename(columns={'company':'instrument','Date':'datetime'},inplace=True)
df_trade['datetime'] = df_trade['datetime'].astype('datetime64[ns]')

In [55]:
df_trade=df_trade[df_trade['datetime']>'2022-12-31']

In [56]:
df_trade.reset_index(inplace=True,drop=True)

In [57]:
df_trade

,instrument,datetime,close,open,high,low,turnover,volume,close_r
0,000009.SZ,2023-01-03,12.22,12.06,12.25,11.91,226093346.0,18635349.0,1.010753
1,000009.SZ,2023-01-04,12.15,12.16,12.30,12.08,223670172.0,18369170.0,0.994272
2,000009.SZ,2023-01-05,12.26,12.18,12.35,12.12,308517291.0,25151630.0,1.009053
3,000009.SZ,2023-01-06,12.28,12.26,12.42,12.19,332819569.0,27004871.0,1.001631
4,000009.SZ,2023-01-09,12.19,12.39,12.39,12.15,272123102.0,22219856.0,0.992671
...,...,...,...,...,...,...,...,...,...
28067,603883.SH,2023-12-25,29.37,29.02,29.58,28.80,101007153.0,3457490.0,1.012061
28068,603883.SH,2023-12-26,29.05,29.26,29.49,28.82,66158144.0,2272587.0,0.989105
28069,603883.SH,2023-12-27,30.08,28.94,30.40,28.80,206474442.0,6939076.0,1.035456
28070,603883.SH,2023-12-28,28.98,30.15,30.20,28.44,264520627.0,9053474.0,0.963431


In [58]:
index_date_unique = df_trade['datetime'].unique().tolist()

In [59]:
#qlib交易策略
path='TRA/output/zz500/transformer_tra/pred.pkl'
f=open(path,'rb')
data=pickle.load(f)
df=pd.DataFrame(data)
#df.reset_index(inplace=True)
#df=df[df['datetime']<'2021-12-30']
df.reset_index(inplace=True)
df.rename(columns={'kdcode':'instrument','dt':'datetime'},inplace=True)
# df.rename(columns={'Company':'instrument','Date':'datetime'},inplace=True)
df_score=pd.DataFrame()
#for date in index_date_unique[:-1]:
for date in index_date_unique:
   # loc=np.where(index_date_unique==date)
    #nextdate_loc=np.where(index_date_unique==date)[0][0]+1
    for i in range(0,len(df)):
        if df['datetime'][i]== date:
            b={'datetime':date,'instrument':df['instrument'][i],'score':df['score'][i]}
            df_score=df_score._append(b,ignore_index=True)
x=pd.merge(df_trade,df_score,on=['datetime','instrument'],how='outer')
trade_date_unique=df_trade['datetime'].unique()
df_return=pd.DataFrame()
#b={'datetime':trade_date_unique[0],'daily_return':0}
#df_return=df_return.append(b,ignore_index=True)
for date in trade_date_unique:
    x_day=x[x['datetime']==date]
    r_day=x_day.nlargest(9,columns='score').close_r.mean()
    r_day=r_day-1
    #r_day-=x_day.nsmallest(9,columns='score').close_r.mean()
    b={'datetime':date,'daily_return':r_day}
    df_return=df_return._append(b,ignore_index=True)

In [60]:
portfolio_df_performance = df_return.set_index(['datetime'])

In [61]:
ARR = (1 + portfolio_df_performance).cumprod()
ARR

,daily_return
datetime,
2023-01-03,1.024722
2023-01-04,1.026788
2023-01-05,1.041130
2023-01-06,1.040629
2023-01-09,1.054733
...,...
2023-12-25,1.101277
2023-12-26,1.097398
2023-12-27,1.105964


In [62]:
portfolio_df_performance

,daily_return
datetime,
2023-01-03,0.024722
2023-01-04,0.002016
2023-01-05,0.013967
2023-01-06,-0.000481
2023-01-09,0.013553
...,...
2023-12-25,-0.008798
2023-12-26,-0.003522
2023-12-27,0.007806


In [63]:
alpha_df_performance = pd.DataFrame()
alpha_df_performance['portfolio_daily_return'] = portfolio_df_performance['daily_return']
alpha_df_performance['portfolio_net_value'] = (alpha_df_performance['portfolio_daily_return'] + 1).cumprod()

net_value_columns = ['portfolio_net_value']

alpha_statistics_df = pd.DataFrame(index=alpha_df_performance[net_value_columns].columns,
                                    columns=["年化收益", "年化波动率", "最大回撤率", "夏普率", "Calmar", "IR"])

# alpha_df_performance.set_index("dt", inplace=True)
alpha_df_performance.index = pd.to_datetime(alpha_df_performance.index)
monthly_statistics_df = alpha_df_performance[net_value_columns].resample('m').last()
monthly_statistics_df = pd.concat([alpha_df_performance[:1][
                                        ['portfolio_net_value']],
                                    monthly_statistics_df])
monthly_statistics_df = monthly_statistics_df.pct_change()
monthly_statistics_df = monthly_statistics_df.dropna()
monthly_statistics_df.index = monthly_statistics_df.index.date
## TODO 补充第一年的数据
yearly_statistics_df = alpha_df_performance[net_value_columns].resample('y').last()
yearly_statistics_df = pd.concat([alpha_df_performance[:1][
                                        ['portfolio_net_value']],
                                    yearly_statistics_df])
yearly_statistics_df = yearly_statistics_df.pct_change()
yearly_statistics_df = yearly_statistics_df.dropna()
yearly_statistics_df.index = yearly_statistics_df.index.date

alpha_statistics_df.loc[:, "年化收益"] = np.mean(
    (alpha_df_performance[net_value_columns].tail(1)) ** (252 / len(alpha_df_performance)) - 1)
alpha_statistics_df.loc[:, "年化波动率"] = np.std(
    alpha_df_performance[net_value_columns] / alpha_df_performance[net_value_columns].shift(1) - 1) * np.sqrt(
    252)
alpha_statistics_df.loc[:, "累积收益"] = np.mean(alpha_df_performance[net_value_columns].tail(1) - 1)
alpha_statistics_df.loc[:, "累积波动率"] = np.std(
    alpha_df_performance[net_value_columns] / alpha_df_performance[net_value_columns].shift(1) - 1)
alpha_statistics_df.loc[:, "最大回撤率"] = np.min(
    (alpha_df_performance[net_value_columns] - alpha_df_performance[net_value_columns].cummax()) /
    alpha_df_performance[net_value_columns].cummax())
alpha_statistics_df.loc[:, "夏普率"] = alpha_statistics_df["年化收益"] / alpha_statistics_df["年化波动率"]
alpha_statistics_df.loc[:, "Calmar"] = alpha_statistics_df["年化收益"] / abs(alpha_statistics_df["最大回撤率"])
alpha_statistics_df.loc[:, "IR"] = np.mean(
    alpha_df_performance[net_value_columns] / alpha_df_performance[net_value_columns].shift(1) - 1) * np.sqrt(
    252) / np.std(alpha_df_performance[net_value_columns] / alpha_df_performance[net_value_columns].shift(1) - 1)

C:\Users\胡逸凡\AppData\Local\Temp\ipykernel_13976\980728215.py:12: FutureWarning: 'm' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_statistics_df = alpha_df_performance[net_value_columns].resample('m').last()
C:\Users\胡逸凡\AppData\Local\Temp\ipykernel_13976\980728215.py:20: FutureWarning: 'y' is deprecated and will be removed in a future version, please use 'YE' instead.
  yearly_statistics_df = alpha_df_performance[net_value_columns].resample('y').last()
C:\Users\胡逸凡\AppData\Roaming\Python\Python310\site-packages\numpy\core\fromnumeric.py:3643: FutureWarning: The behavior of DataFrame.std with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return std(axis=axis, dtype=dtype, out=out, ddof=ddof, **kwargs)
C:\Users\胡逸凡\AppData\Roaming\Python\Python310\site-packages\numpy\core\fromnumeric.py:3643: FutureWarning: The behavior of DataFra

In [64]:
alpha_statistics_df

,年化收益,年化波动率,最大回撤率,夏普率,Calmar,IR,累积收益,累积波动率
portfolio_net_value,0.12558,0.161882,-0.144962,0.775749,0.866293,0.656948,0.120308,0.010198


ASR = ARR / AVol;   CR = ARR / abs(MDD)
                        ARR    AVol    MDD    ASR     CR    IR
hs300  lstm            -0.214  0.157  -0.275 -1.361 -0.779 -1.492
       lstm tra        -0.216  0.164  -0.294 -1.314 -0.735 -1.461
       gru             -0.229  0.156  -0.290 -1.469 -0.790 -1.631
       transformer     -0.240  0.156  -0.281 -1.543 -0.855 -1.695
       transformer tra -0.074  0.169  -0.222 -0.436 -0.332 -0.409
      
zz500  lstm            -0.008  0.159  -0.172 -0.047 -0.044 -0.128
       lstm tra         0.016  0.162  -0.192  0.101  0.086  0.014
       gru             -0.004  0.159  -0.193 -0.028 -0.023 -0.118
       transformer      0.154  0.156  -0.135  0.986  1.143  0.867
       transformer tra  0.125  0.162  -0.145  0.776  0.866  0.657
      
sp500  lstm            0.142  0.162  -0.178  0.877  0.798  0.929
       lstm tra        0.191  0.161  -0.150  1.186  1.273  1.115
       gru             0.324  0.169  -0.139  1.917  2.331  1.657
       transformer     0.135  0.159  -0.140  0.852  0.968  0.908
       transformer tra 0.184  0.166  -0.158  1.114  1.172  1.106
      
nas100 lstm            0.247  0.176  -0.128  1.403  1.930  1.386
       lstm tra        0.201  0.192  -0.183  1.047  1.098  1.032
       gru             0.225  0.188  -0.165  1.197  1.364  1.160
       transformer     0.268  0.175  -0.131  1.531  2.046  1.441
       transformer tra 0.267  0.181  -0.144  1.475  1.854  1.427
       